In [1]:
import torch
import torchvision
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import cv2
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
from datetime import datetime
import numpy as np
import albumentations as A

In [2]:
# dataset definition
class myDataset(Dataset):
    # load the dataset
    def __init__(self):
        # store the inputs and outputs
        self.data = pd.read_csv("/home/semone/notebook/soft/annotations.csv")
        self.image = self.data.iloc[:,0]
        self.targets = {'boxes':self.data.iloc[:,1:5].values, 'labels': self.data.iloc[:,5].values}
        #self.label = 

    # number of rows in the dataset
    def __len__(self):
        return len(self.data)

    # get a row at an index
    def __getitem__(self, idx):
        box = self.targets['boxes'][idx]
        label = self.targets['labels'][idx]
        com = {'boxes':box, 'labels': label}
        
        return [self.image[idx], com]

In [3]:
ds = myDataset()

In [4]:
ds.__getitem__(1)

['sample_002.png', {'boxes': array([  4, 246,  43, 283]), 'labels': 1}]

In [5]:
transform = A.Compose(
    [A.Resize(height=300, width=300)],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']),
)

In [6]:
transformT = transforms.Compose([
    transforms.ToTensor()])

In [7]:
category_ids = [0]
category_id_to_name = {0: 'stop'}
for x in range(900):
    image = cv2.imread(ds.image[x])
    bboxes = [ds.targets['boxes'][x]]
    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
    toimg = transformed['image']
    tt= transformT(toimg)
    ds.image[x] = tt
    
    k =transformed['bboxes']
    ds.targets['boxes'][x] = k[0]
    
    #label = ds.target['labels'][x]
    #dic = {'boxes':k,'labels': label}
    #ds.target[x] = dic

/tmp/ipykernel_23126/1234328497.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds.image[x] = tt


richtig skaliert und totensort - model

In [64]:
train_dataloader = DataLoader(ds, batch_size=2)
test_dataloader = DataLoader(ds, batch_size=2)

In [21]:
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn()
#@todo Fix Hardcoded params
parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
            parameters,
            lr=0.01,
            momentum=1,
            weight_decay=1,
        )

In [23]:
for i in range(100):
    for image, targets in train_dataloader:
        result = model(image, targets)
    #with torch.cuda.amp.autocast(enabled=scaler is not None):
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

TypeError: string indices must be integers